# Chapter 6

## Data Loading, Storage, and File Formats

### 6.1 Reading and Writing Data in Text Format
Mecanics of reading tabular data functions<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Indexing<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Type inference and data conversion : some formats do not have data type in the format, some do<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Datetime parsing<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Iterating<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Unclean data issues

In [1]:
import pandas as pd
import numpy as np

In [2]:
!cat '../pydata-book-2nd-edition/examples/ex1.csv'

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
examples = '../pydata-book-2nd-edition/examples/'

In [4]:
datasets = '../pydata-book-2nd-edition/datasets/'

In [5]:
examples

'../pydata-book-2nd-edition/examples/'

In [6]:
df = pd.read_csv(examples+'ex1.csv')

In [7]:
df

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [8]:
!cat '../pydata-book-2nd-edition/examples/ex2.csv' # when a file had no header row

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


cat: '#': No such file or directory
cat: when: No such file or directory
cat: a: No such file or directory
cat: file: No such file or directory
cat: had: No such file or directory
cat: no: No such file or directory
cat: header: No such file or directory
cat: row: No such file or directory


In [9]:
pd.read_csv(examples+'ex2.csv', header=None)

0   1   2   3      4
0  1   2   3   4  hello
1  5   6   7   8  world
2  9  10  11  12    foo

In [10]:
pd.read_csv(examples+'ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

Suppose you wanted the message column to be the index

In [11]:
names = ['a', 'b', 'c', 'd', 'message']

In [12]:
pd.read_csv(examples+'ex2.csv', names=names, index_col='message')

a   b   c   d
message               
hello    1   2   3   4
world    5   6   7   8
foo      9  10  11  12

Hierarchical index

In [13]:
!cat '../pydata-book-2nd-edition/examples/csv_mindex.csv'

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [14]:
parsed = pd.read_csv(examples+'csv_mindex.csv',
                    index_col=['key1', 'key2'])

In [15]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [16]:
list(open(examples+'ex3.txt')) # some files migth no have a fixed delimiter

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

In [17]:
result = pd.read_csv(examples+'ex3.txt', sep='\s+')

In [18]:
result # first column is auto DataFrame index because one less col than rows in this special case

A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491

In [19]:
!cat ../pydata-book-2nd-edition/examples/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [20]:
pd.read_csv(examples+'ex4.csv', skiprows=[0, 2, 3])

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [21]:
!cat ../pydata-book-2nd-edition/examples/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [22]:
result = pd.read_csv(examples+'ex5.csv') # handeling missing data automaticaly

In [23]:
result

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

In [24]:
pd.isnull(result)

something      a      b      c      d  message
0      False  False  False  False  False     True
1      False  False  False   True  False    False
2      False  False  False  False  False    False

In [25]:
result = pd.read_csv(examples+'ex5.csv', na_values=['NULL']) # na_values option

In [26]:
result

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

In [27]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']} # different NA sentinels for each col via dict

In [28]:
pd.read_csv(examples+'ex5.csv', na_values=sentinels)

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN

#### Reading Text Files in Pieces

In [29]:
pd.options.display.max_rows = 10 # making the pandas display more compact

In [30]:
results = pd.read_csv(examples+'ex6.csv')

In [31]:
results

one       two     three      four key
0     0.467976 -0.038649 -0.295344 -1.824726   L
1    -0.358893  1.404453  0.704965 -0.200638   B
2    -0.501840  0.659254 -0.421691 -0.057688   G
3     0.204886  1.074134  1.388361 -0.982404   R
4     0.354628 -0.133116  0.283763 -0.837063   Q
...        ...       ...       ...       ...  ..
9995  2.311896 -0.417070 -1.409599 -0.515821   L
9996 -0.479893 -0.650419  0.745152 -0.646038   E
9997  0.523331  0.787112  0.486066  1.093156   K
9998 -0.362559  0.598894 -1.843201  0.887292   G
9999 -0.096376 -1.012999 -0.657431 -0.573315   0

[10000 rows x 5 columns]

In [32]:
pd.read_csv(examples+'ex6.csv', nrows=5) # read the first five rows

one       two     three      four key
0  0.467976 -0.038649 -0.295344 -1.824726   L
1 -0.358893  1.404453  0.704965 -0.200638   B
2 -0.501840  0.659254 -0.421691 -0.057688   G
3  0.204886  1.074134  1.388361 -0.982404   R
4  0.354628 -0.133116  0.283763 -0.837063   Q

In [33]:
chunker = pd.read_csv(examples+'ex6.csv', chunksize=1000) # read a file in pieces

In [34]:
chunker

Iterate over ex6.csv, aggrigating the value counts in the 'key' column

In [35]:
tot = pd.Series([])

In [36]:
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

In [37]:
tot = tot.sort_values(ascending=False)

In [38]:
tot[:10] # display the top 10 totals

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

#### Writing Data to Text Format

In [39]:
data = pd.read_csv(examples+'ex5.csv')

In [40]:
data

something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       two  5   6   NaN   8   world
2     three  9  10  11.0  12     foo

In [41]:
data.to_csv(examples+'out.csv') # using DataFrame's to_csv method to write/overwrite to csv file

In [42]:
!cat ../pydata-book-2nd-edition/examples/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


In [43]:
import sys # writing to sys.stdout so it prints the text to the console
data.to_csv(sys.stdout, sep='|') # other delimiters can be used

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


In [44]:
data.to_csv(sys.stdout, na_rep='NULL') # denote missing values with NULL

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


In [45]:
data.to_csv(sys.stdout, index=False, header=False) # disable col and row labels

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


In [46]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a to_csv method

In [47]:
dates = pd.date_range('1/1/2000', periods=7)

In [48]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')

In [49]:
 ts = pd.Series(np.arange(7), index=dates) # np.arange create an array ([0 - 6])

In [50]:
ts.to_csv(examples+'tseries.csv')

c:\users\martinvm1\envs\py2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [51]:
!cat ../pydata-book-2nd-edition/examples/tseries.csv

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


#### Working with Delimited Formats

In [52]:
!cat ../pydata-book-2nd-edition/examples/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


In [53]:
import csv
f = open(examples+'ex7.csv')

In [54]:
reader = csv.reader(f) # pass any open file or file-like object to csv.reader

In [55]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


Wrangling the data, spliting the header line from the data lines

In [56]:
with open(examples+'ex7.csv') as f:
    lines = list(csv.reader(f))

In [57]:
header, values = lines[0], lines[1:] # header first line, remaining lines are values

Then we can create a dict and the expression zip(*values) which transposes rows to columns

In [58]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [59]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV dialect

In [60]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [61]:
f = open(examples+'ex7.csv') # extra to book, file was closed before I got here

In [62]:
reader = csv.reader(f, dialect=my_dialect)

In [63]:
reader = csv.reader(f, delimiter='|') # we can give CSV dialect parameters as keywords without having to define a sub-class

To write delimited files manually

In [64]:
with open(examples+'mydata.csv', 'w') as f:
    write = csv.writer(f, dialect=my_dialect)
    write.writerow(('one', 'two', 'three'))
    write.writerow(('1', '2', '3'))
    write.writerow(('4', '5', '6'))
    write.writerow(('7', '8', '9'))

#### JSON Data

In [65]:
obj = """
{"name": "Wes",
"place_lived": ["United States", "Spain", "Germany"],
"pet": null,
"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
             {"name": "Katie", "age": 38,
                "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

In [66]:
obj

'\n{"name": "Wes",\n"place_lived": ["United States", "Spain", "Germany"],\n"pet": null,\n"siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},\n             {"name": "Katie", "age": 38,\n                "pets": ["Sixes", "Stache", "Cisco"]}]\n}\n'

In [67]:
import json

In [68]:
result = json.loads(obj)

In [69]:
result

{'name': 'Wes',
 'place_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

In [70]:
asjson = json.dumps(result) # convert the python object back to json

You can pass a list of dicts (previous JSON objects) to the DataFrame constructor and select a sub-set of the data

In [71]:
siblings = pd.DataFrame(result['siblings'], columns=['name', 'age']) # 

In [72]:
siblings

name  age
0  Scott   30
1  Katie   38

pandas.read_json can automatically convert JSON datasets into a Series of DataFrame

In [73]:
!cat ../pydata-book-2nd-edition/examples/example.json

[{"a": 1, "b": 2, "c": 3},
 {"a": 4, "b": 5, "c": 6},
 {"a": 7, "b": 8, "c": 9}]


In [74]:
data = pd.read_json(examples+'example.json') # default assumes each object is the JSON array in a row in the table

In [75]:
data

a  b  c
0  1  2  3
1  4  5  6
2  7  8  9

In [76]:
print(data.to_json()) # print/export option 1

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}


In [77]:
print(data.to_json(orient='records')) # print/export option 2

[{"a":1,"b":2,"c":3},{"a":4,"b":5,"c":6},{"a":7,"b":8,"c":9}]


#### XML and HTML: Web Scraping
Useful libraries are lxml (http://lxml.de), Beautiful Soup and html5lib, lxml is the fastest<br/>
installed beautifulsoup4 & html5lib

pandas.read_html by default searches and attempts to parse all tabular data contained within < table > tags resulting in DataFrame objects<br/>

In [78]:
tables = pd.read_html(examples+'fdic_failed_bank_list.html')

In [79]:
len(tables)

1

In [80]:
failures = tables[0]

In [81]:
failures.head()

Bank Name             City  ST   CERT  \
0                   Allied Bank         Mulberry  AR     91   
1  The Woodbury Banking Company         Woodbury  GA  11297   
2        First CornerStone Bank  King of Prussia  PA  35312   
3            Trust Company Bank          Memphis  TN   9956   
4    North Milwaukee State Bank        Milwaukee  WI  20364   

                 Acquiring Institution        Closing Date       Updated Date  
0                         Today's Bank  September 23, 2016  November 17, 2016  
1                          United Bank     August 19, 2016  November 17, 2016  
2  First-Citizens Bank & Trust Company         May 6, 2016  September 6, 2016  
3           The Bank of Fayette County      April 29, 2016  September 6, 2016  
4  First-Citizens Bank & Trust Company      March 11, 2016      June 16, 2016

In [82]:
close_timestamp = pd.to_datetime(failures['Closing Date'])

In [83]:
close_timestamp.dt.year.value_counts()

2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: Closing Date, Length: 15, dtype: int64

##### Parsing XML with lxml.objectify

In [84]:
from lxml import objectify

In [85]:
path = datasets+'mta_perf/Performance_MNR.xml'

In [86]:
parsed = objectify.parse(open(path))

In [87]:
root = parsed.getroot()

In [88]:
data = []

In [89]:
skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ', 'DESIRED_CHANGE', 'DECIMAL_PLACE']

In [90]:
for etl in root.INDICATOR:
    el_data = {}
    for child in etl.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

In [91]:
pref = pd.DataFrame(data)

In [92]:
pref.head()

AGENCY_NAME            CATEGORY  DECIMAL_PLACES  \
0  Metro-North Railroad  Service Indicators               1   
1  Metro-North Railroad  Service Indicators               1   
2  Metro-North Railroad  Service Indicators               1   
3  Metro-North Railroad  Service Indicators               1   
4  Metro-North Railroad  Service Indicators               1   

                                         DESCRIPTION FREQUENCY  \
0  Percent of commuter trains that arrive at thei...         M   
1  Percent of commuter trains that arrive at thei...         M   
2  Percent of commuter trains that arrive at thei...         M   
3  Percent of commuter trains that arrive at thei...         M   
4  Percent of commuter trains that arrive at thei...         M   

                         INDICATOR_NAME INDICATOR_UNIT MONTHLY_ACTUAL  \
0  On-Time Performance (West of Hudson)              %           96.9   
1  On-Time Performance (West of Hudson)              %             95   
2  On-Time Performance (West of Hudson)              %           96.9   
3  On-Time Performance (West of Hudson)              %           98.3   
4  On-Time Performance (West of Hudson)              %           95.8   

  MONTHLY_TARGET  PERIOD_MONTH  PERIOD_YEAR YTD_ACTUAL YTD_TARGET  
0             95             1         2008       96.9         95  
1             95             2         2008         96         95  
2             95             3         2008       96.3         95  
3             95             4         2008       96.8         95  
4             95             5         2008       96.6         95

Another example : consider a HTML link tag

In [93]:
from io import StringIO

In [94]:
tag = '<a href="http://www.google.com">Google</a>'

In [95]:
root = objectify.parse(StringIO(tag)).getroot()

In [96]:
root

<Element a at 0x1d77dd93b48>

In [97]:
root.get('href')

'http://www.google.com'

In [98]:
root.text

'Google'

### Binary Data Formats

In [99]:
frame = pd.read_csv(examples+'ex1.csv')

In [100]:
frame

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

In [101]:
frame.to_pickle(examples+'frame_pickle')

In [102]:
pd.read_pickle(examples+'frame_pickle')

a   b   c   d message
0  1   2   3   4   hello
1  5   6   7   8   world
2  9  10  11  12     foo

pandas has built-in support for two more binary formats: HDF5 and Message-Pack

other formats to consider, depending on you analytics are bcolz and Feather

#### Using HDF5 Format
Hierarchical data format<br/>
Can access directly using PyTables or h5py, however HDFStore simplifies this, works like a dict and handles the low-level details

In [103]:
frame = pd.DataFrame({'a': np.random.rand(100)})

pip install tables

In [104]:
store = pd.HDFStore(examples+'mydata.h5')

In [105]:
store['obj1'] = frame

In [106]:
store['obj1_col'] = frame['a']

In [107]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: ../pydata-book-2nd-edition/examples/mydata.h5

In [108]:
store['obj1']

a
0   0.946690
1   0.745955
2   0.723117
3   0.744815
4   0.382292
..       ...
95  0.359988
96  0.436516
97  0.279989
98  0.594985
99  0.115975

[100 rows x 1 columns]

HDFStore storage schemas are 'fixed' and 'table'. table is slower but queryable

In [109]:
store.put('obj2', frame, format='table') # the file mydata.h5 was modified, probably contains obj2 on disk

In [110]:
store.select('obj2', where=['index >= 10 and index <= 15'])

a
10  0.840907
11  0.926885
12  0.743885
13  0.146401
14  0.617165
15  0.427603

In [111]:
store.close()

In [112]:
frame.to_hdf(examples+'mydata.h5', 'obj3' , format='table')

In [113]:
pd.read_hdf(examples+'mydata.h5', 'obj3', where=['index < 5'])

a
0  0.946690
1  0.745955
2  0.723117
3  0.744815
4  0.382292

#### Reading Excel Files
using ExcelFile class or pandas.read_excel functions<br/>
pip install xlrd and openpyxl

In [114]:
xlsx = pd.ExcelFile(examples+'ex1.xlsx') # excel read option 1

In [115]:
pd.read_excel(xlsx, 'Sheet1')

Unnamed: 0  a   b   c   d message
0           0  1   2   3   4   hello
1           1  5   6   7   8   world
2           2  9  10  11  12     foo

In [116]:
frame = pd.read_excel(examples+'ex1.xlsx', 'Sheet1') # excel read option 2

In [117]:
frame

Unnamed: 0  a   b   c   d message
0           0  1   2   3   4   hello
1           1  5   6   7   8   world
2           2  9  10  11  12     foo

ExcelWriter

In [118]:
writer = pd.ExcelWriter(examples+'ex2.xlsx')

In [119]:
frame.to_excel(writer, 'Sheet1')

In [120]:
writer.save()

You can also pass a filename to .to_excel and avoid the ExcelWriter

In [121]:
frame.to_excel(examples+'ex2.xlsx')

read back

In [122]:
xlsx = pd.ExcelFile(examples+'ex2.xlsx') # excel read option 1

In [123]:
pd.read_excel(xlsx, 'Sheet1')

Unnamed: 0  Unnamed: 0.1  a   b   c   d message
0           0             0  1   2   3   4   hello
1           1             1  5   6   7   8   world
2           2             2  9  10  11  12     foo

### Interacting with Web APIs
Some websites have public APIs providing data feeds via JSON or some other firmat. requests package recommended

Find the last 30 GitHub issues for Pandas on GitHub

In [124]:
import requests

In [125]:
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'

In [126]:
resp = requests.get(url)

In [127]:
resp

<Response [200]>

The Response object's json method will return a dictionary containing JSON parsed into native Python objects

In [128]:
data = resp.json()

In [129]:
data[0]['title']

'Initial pandas.typing Module'

In [130]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state'])

In [131]:
issues

number                                              title  \
0    25884                       Initial pandas.typing Module   
1    25883  GroupBy.apply runtime performance regression i...   
2    25882          Typing Cleanup - Remove Blacklisted Items   
3    25881       .at does not work using a categorical index.   
4    25880  Int64Dtype conversion seems to casts to double...   
..     ...                                                ...   
25   25823  Logical operations on np.bool_ using pandas.ev...   
26   25820  [PERF] Get rid of MultiIndex conversion in Int...   
27   25818  sort_index for MultiIndex DataFrame silently f...   
28   25816  Series.replace throws TypeError only on large ...   
29   25815  wrong aggregated results when pivot_table inde...   

                                               labels state  
0   [{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...  open  
1   [{'id': 8935311, 'node_id': 'MDU6TGFiZWw4OTM1M...  open  
2   [{'id': 1280988427, 'node_id': 'MDU6TGFiZWwxMj...  open  
3                                                  []  open  
4                                                  []  open  
..                                                ...   ...  
25  [{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...  open  
26  [{'id': 150096370, 'node_id': 'MDU6TGFiZWwxNTA...  open  
27                                                 []  open  
28                                                 []  open  
29  [{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...  open  

[30 rows x 4 columns]

### Interacting with Databases

sqlite3

In [132]:
import sqlite3

In [133]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20), c REAL, d INTERGER);
"""

In [134]:
con = sqlite3.connect('mydata.sqlite')

In [135]:
con.execute(query)

OperationalError: table test already exists

In [ ]:
con.commit()

In [ ]:
data = [('Atlanta', 'Georgia', 1.25, 6),
       ('Tallahasee', 'Florida', 2.6, 3),
       ('Sacramento', 'California', 1.7, 5)]

In [ ]:
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

In [ ]:
con.executemany(stmt, data)

In [ ]:
cursor = con.execute('select * from test')

In [ ]:
rows = cursor.fetchall()

In [ ]:
rows

In [ ]:
cursor.description

In [ ]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

SQLAlchemy abstracts away many of the common differences between SQL databases

In [ ]:
# import sqlalchemy as sqla # I have decided not to use SQLAlchemy for now

MySQL

In [136]:
import mysql.connector

In [143]:
mydb = mysql.connector.connect(host="localhost", user="root", passwd="MySQLsad1", database="world")

In [144]:
mycursor = mydb.cursor()

In [149]:
mycursor.execute("select * from city")

In [150]:
result = mycursor.fetchall()

In [152]:
for i in result:
    print(i)

(1, 'Kabul', 'AFG', 'Kabol', 1780000)
(2, 'Qandahar', 'AFG', 'Qandahar', 237500)
(3, 'Herat', 'AFG', 'Herat', 186800)
(4, 'Mazar-e-Sharif', 'AFG', 'Balkh', 127800)
(5, 'Amsterdam', 'NLD', 'Noord-Holland', 731200)
(6, 'Rotterdam', 'NLD', 'Zuid-Holland', 593321)
(7, 'Haag', 'NLD', 'Zuid-Holland', 440900)
(8, 'Utrecht', 'NLD', 'Utrecht', 234323)
(9, 'Eindhoven', 'NLD', 'Noord-Brabant', 201843)
(10, 'Tilburg', 'NLD', 'Noord-Brabant', 193238)
(11, 'Groningen', 'NLD', 'Groningen', 172701)
(12, 'Breda', 'NLD', 'Noord-Brabant', 160398)
(13, 'Apeldoorn', 'NLD', 'Gelderland', 153491)
(14, 'Nijmegen', 'NLD', 'Gelderland', 152463)
(15, 'Enschede', 'NLD', 'Overijssel', 149544)
(16, 'Haarlem', 'NLD', 'Noord-Holland', 148772)
(17, 'Almere', 'NLD', 'Flevoland', 142465)
(18, 'Arnhem', 'NLD', 'Gelderland', 138020)
(19, 'Zaanstad', 'NLD', 'Noord-Holland', 135621)
(20, 'Â´s-Hertogenbosch', 'NLD', 'Noord-Brabant', 129170)
(21, 'Amersfoort', 'NLD', 'Utrecht', 126270)
(22, 'Maastricht', 'NLD', 'Limburg', 122

(1111, 'Gaya', 'IND', 'Bihar', 291675)
(1112, 'Kakinada', 'IND', 'Andhra Pradesh', 279980)
(1113, 'Dhule (Dhulia)', 'IND', 'Maharashtra', 278317)
(1114, 'Panihati', 'IND', 'West Bengali', 275990)
(1115, 'Nanded (Nander)', 'IND', 'Maharashtra', 275083)
(1116, 'Mangalore', 'IND', 'Karnataka', 273304)
(1117, 'Dehra Dun', 'IND', 'Uttaranchal', 270159)
(1118, 'Kamarhati', 'IND', 'West Bengali', 266889)
(1119, 'Davangere', 'IND', 'Karnataka', 266082)
(1120, 'Asansol', 'IND', 'West Bengali', 262188)
(1121, 'Bhagalpur', 'IND', 'Bihar', 253225)
(1122, 'Bellary', 'IND', 'Karnataka', 245391)
(1123, 'Barddhaman (Burdwan)', 'IND', 'West Bengali', 245079)
(1124, 'Rampur', 'IND', 'Uttar Pradesh', 243742)
(1125, 'Jalgaon', 'IND', 'Maharashtra', 242193)
(1126, 'Muzaffarpur', 'IND', 'Bihar', 241107)
(1127, 'Nizamabad', 'IND', 'Andhra Pradesh', 241034)
(1128, 'Muzaffarnagar', 'IND', 'Uttar Pradesh', 240609)
(1129, 'Patiala', 'IND', 'Punjab', 238368)
(1130, 'Shahjahanpur', 'IND', 'Uttar Pradesh', 237713)


(1932, 'Yichun', 'CHN', 'Heilongjiang', 800000)
(1933, 'Benxi', 'CHN', 'Liaoning', 770000)
(1934, 'Luoyang', 'CHN', 'Henan', 760000)
(1935, 'Suzhou', 'CHN', 'Jiangsu', 710000)
(1936, 'Xining', 'CHN', 'Qinghai', 700200)
(1937, 'Huainan', 'CHN', 'Anhui', 700000)
(1938, 'Jixi', 'CHN', 'Heilongjiang', 683885)
(1939, 'Daqing', 'CHN', 'Heilongjiang', 660000)
(1940, 'Fuxin', 'CHN', 'Liaoning', 640000)
(1941, 'Amoy [Xiamen]', 'CHN', 'Fujian', 627500)
(1942, 'Liuzhou', 'CHN', 'Guangxi', 610000)
(1943, 'Shantou', 'CHN', 'Guangdong', 580000)
(1944, 'Jinzhou', 'CHN', 'Liaoning', 570000)
(1945, 'Mudanjiang', 'CHN', 'Heilongjiang', 570000)
(1946, 'Yinchuan', 'CHN', 'Ningxia', 544500)
(1947, 'Changzhou', 'CHN', 'Jiangsu', 530000)
(1948, 'Zhangjiakou', 'CHN', 'Hebei', 530000)
(1949, 'Dandong', 'CHN', 'Liaoning', 520000)
(1950, 'Hegang', 'CHN', 'Heilongjiang', 520000)
(1951, 'Kaifeng', 'CHN', 'Henan', 510000)
(1952, 'Jiamusi', 'CHN', 'Heilongjiang', 493409)
(1953, 'Liaoyang', 'CHN', 'Liaoning', 492559)

(2953, 'Walbrzych', 'POL', 'Dolnoslaskie', 136923)
(2954, 'Tychy', 'POL', 'Slaskie', 133178)
(2955, 'Dabrowa GÃ³rnicza', 'POL', 'Slaskie', 131037)
(2956, 'Plock', 'POL', 'Mazowieckie', 131011)
(2957, 'Elblag', 'POL', 'Warminsko-Mazurskie', 129782)
(2958, 'Opole', 'POL', 'Opolskie', 129553)
(2959, 'GorzÃ³w Wielkopolski', 'POL', 'Lubuskie', 126019)
(2960, 'Wloclawek', 'POL', 'Kujawsko-Pomorskie', 123373)
(2961, 'ChorzÃ³w', 'POL', 'Slaskie', 121708)
(2962, 'TarnÃ³w', 'POL', 'Malopolskie', 121494)
(2963, 'Zielona GÃ³ra', 'POL', 'Lubuskie', 118182)
(2964, 'Koszalin', 'POL', 'Zachodnio-Pomorskie', 112375)
(2965, 'Legnica', 'POL', 'Dolnoslaskie', 109335)
(2966, 'Kalisz', 'POL', 'Wielkopolskie', 106641)
(2967, 'Grudziadz', 'POL', 'Kujawsko-Pomorskie', 102434)
(2968, 'Slupsk', 'POL', 'Pomorskie', 102370)
(2969, 'Jastrzebie-ZdrÃ³j', 'POL', 'Slaskie', 102294)
(2970, 'Jaworzno', 'POL', 'Slaskie', 97929)
(2971, 'Jelenia GÃ³ra', 'POL', 'Dolnoslaskie', 93901)
(2972, 'Malabo', 'GNQ', 'Bioko', 40000)
(

(4039, 'Midland', 'USA', 'Texas', 98293)
(4040, 'Davenport', 'USA', 'Iowa', 98256)
(4041, 'Mission Viejo', 'USA', 'California', 98049)
(4042, 'Miami Beach', 'USA', 'Florida', 97855)
(4043, 'Sunrise Manor', 'USA', 'Nevada', 95362)
(4044, 'New Bedford', 'USA', 'Massachusetts', 94780)
(4045, 'El Cajon', 'USA', 'California', 94578)
(4046, 'Norman', 'USA', 'Oklahoma', 94193)
(4047, 'Richmond', 'USA', 'California', 94100)
(4048, 'Albany', 'USA', 'New York', 93994)
(4049, 'Brockton', 'USA', 'Massachusetts', 93653)
(4050, 'Roanoke', 'USA', 'Virginia', 93357)
(4051, 'Billings', 'USA', 'Montana', 92988)
(4052, 'Compton', 'USA', 'California', 92864)
(4053, 'Gainesville', 'USA', 'Florida', 92291)
(4054, 'Fairfield', 'USA', 'California', 92256)
(4055, 'Arden-Arcade', 'USA', 'California', 92040)
(4056, 'San Mateo', 'USA', 'California', 91799)
(4057, 'Visalia', 'USA', 'California', 91762)
(4058, 'Boulder', 'USA', 'Colorado', 91238)
(4059, 'Cary', 'USA', 'North Carolina', 91213)
(4060, 'Santa Monica',